# Tutorial 9 - Simple Reason-Act Agent from Scratch 
See more at https://react-lm.github.io/

**Course:** SYSC 4415 - Introduction to Machine Learning

**Semester:** Winter 2025

**Recreated by:** [Igor Bogdanov](mailto:igorbogdanov@cmailcarleton.ca) 

**Inspired by:** [NVIDIA GTC 2025](https://www.nvidia.com/gtc/) Agentic Session

---

This notebook demonstrates key concepts in building a basic ReAct agent:

---
1. **AI Agent Core Components**
   - Essential elements that compose an intelligent agent
   - Prompt engineering and message processing techniques
   - Conversation state and dialogue management
---
2. **Reasoning-Action Framework**
   - Cognitive process cycle: Thought → Action → Observation → Answer cycle
   - Structured reasoning process
   - Tool utilization and output interpretation
---
3. **Tool Orchestration**
   - Creating callable functions (model_memory, apply_conversion)
   - Processing external data and maintaining system state
---
4. **Applied Agent Development**
   - Case study: Measurement Conversion System
   - Multi-step problem decomposition
   - Automated reasoning with integrated capabilities
---

<img src="assets/react_agent_flowchart.png"  width="20%;" style="background-color:Gainsboro; padding:5%;"/>

In [58]:
import os
from typing import Dict, List, Optional
from dataclasses import dataclass
import re
from dotenv import load_dotenv
from groq import Groq


In [59]:
client = Groq(
api_key=os.environ.get("GROQ_API_KEY", "API_KEY_VALUE"))


In [60]:

# Update the initial test completion
chat_completion = client.chat.completions.create(
    model = "llama-3.3-70b-versatile", 
    messages=[{"role": "user", "content": "Hello! What is happening?"}],
    temperature=0.2,
    top_p=0.7,
    max_tokens=1024
)

chat_completion.choices[0].message.content

"Hello. I'm here to help with any questions or topics you'd like to discuss. It seems like we've just started our conversation, and I'm not aware of any specific events or situations you might be referring to. Could you please provide more context or information about what you're asking? I'll do my best to help."

In [61]:
@dataclass
class AgentState:
    """Represents the current state of the agent."""

    messages: List[Dict[str, str]]
    system_prompt: str


class ReActAgent:
    """
    A ReAct agent specialized in code analysis that follows the Thought -> Action -> Observation -> Answer pattern.

    This implementation demonstrates:
    1. Code complexity analysis
    2. Dependency detection
    3. Code quality assessment
    4. Improvement suggestions
    """

    def __init__(self, system_prompt: str):
        """
        Initialize the code analysis agent.

        Args:
            system_prompt: The system prompt that defines the agent's behavior
        """
        self.client = client
        self.state = AgentState(
            messages=[{"role": "system", "content": system_prompt}],
            system_prompt=system_prompt,
        )

    def add_message(self, role: str, content: str) -> None:
        """Add a message to the agent's conversation history."""
        self.state.messages.append({"role": role, "content": content})

    def execute(self) -> str:
        """
        Execute the agent's next action based on the current state.

        Returns:
            The agent's response
        """
        completion = self.client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            temperature=0.2,
            top_p=0.7,
            max_tokens=1024,
            messages=self.state.messages,
        )
        return completion.choices[0].message.content

    def __call__(self, message: str) -> str:
        """
        Process a user message and return the agent's response.

        Args:
            message: The user's input message

        Returns:
            The agent's response
        """
        self.add_message("user", message)
        result = self.execute()
        self.add_message("assistant", result)
        return result  

In [72]:
# Tools

def model_memory(unit: str) -> str:
    """
    Get the conversion rate for a given unit.

    Args:
        unit: The unit to get conversion rate for

    Returns:
        The conversion rate as a numerical value
    """
    rates = {
        "meters to feet": "3.28084",
        "kilometers to miles": "0.621371",
        "kilograms to pounds": "2.20462",
        "celsius to fahrenheit": "9/5,32",  # Special case with two parameters
    }
    return rates.get(unit.lower(), f"No conversion rate found for {unit}")

def apply_conversion(rate: str) -> str:
    """
    Apply a conversion rate to a value.

    Args:
        rate: The conversion rate obtained from get_conversion_rate
        value: The value to convert

    Returns:
        The converted value
    """
    try:
        # Split the input string into rate and value
        params = rate.split(",")
        if len(params) == 3:
            # Handle temperature conversion (multiplier, offset, value)
            multiplier, offset, value_str = params
            num = float(value_str.strip())
            # Evaluate the fraction for temperature conversion
            multiplier = eval(multiplier.strip())  # This will evaluate "9/5" to 1.8
            offset = float(offset.strip())
            result = num * multiplier + offset
            return f"{num}°C = {result:.2f}°F"
        elif len(params) == 2:
            # Handle regular conversion (rate, value)
            rate_str, value_str = params
            num = float(value_str.strip())
            conversion_rate = float(rate_str)
            result = num * conversion_rate
            # Map conversion rates to their source and target units
            unit_map = {
                "3.28084": ("meters", "feet"),
                "0.621371": ("kilometers", "miles"),
                "2.20462": ("kilograms", "pounds"),
            }
            source_unit, target_unit = unit_map.get(rate_str, ("units", "units"))
            return f"{num} {source_unit} = {result:.2f} {target_unit}"
        else:
            return "Error: Invalid number of parameters"
    except ValueError:
        return "Error: Please provide valid numbers"
    except SyntaxError:
        return "Error: Invalid conversion rate format"

In [73]:
def create_agent() -> ReActAgent:
    """
    Create a conversion agent with the tutorial system prompt.
    """
    system_prompt = """
    You run in a loop of Thought, Action, PAUSE, Observation.
    At the end of the loop you output an Answer
    Use Thought to describe your thoughts about the question you have been asked.
    Use Action to run one of the actions available to you - then return PAUSE.
    Observation will be the result of running those actions.
    
    Your available actions are:
    
    model_memory:
    e.g. model_memory: meters to feet
    Get the conversion rate for a unit (e.g., meters to feet, kilometers to miles)

    apply_conversion: 3.28084, 5
    Applies the conversion rate to a value and returns a number
    
    No other actions are available to you.
    
    Example session #1:
    Question: Convert 5 meters to feet?
    
    Thought: This is a length conversion from meters to feet. First, I need to get the conversion rate
    Action: get_conversion_rate: meters to feet
    PAUSE

    You will be called again with this:
    
    Observation: 3.28084

    You then output:
    
    Thought: Now I can apply this conversion rate to 5 meters
    Action: apply_conversion: 3.28084, 5
    PAUSE
    
    Observation: 5 meters = 16.40 feet
    
    Answer: 5 meters is equal to 16.40 feet  

    Example session #2:

    Question:  Convert 20°C to Fahrenheit?
    
    Thought: This is a temperature conversion from Celsius to Fahrenheit. First, I need to get the conversion formula
    Action: model_memory: celsius to fahrenheit
    PAUSE

    You will be called again with this:
    
    Observation: 9/5,32

    You then output:
    
    Thought: Now I can apply this formula to 20°C
    Action: apply_conversion: 9/5,32, 20
    PAUSE
    
    Observation: 20°C = 68.00°F
    
    Answer: 20°C is equal to 68.00°F 

    
    """.strip()

    return ReActAgent(system_prompt)


In [74]:
agent = create_agent()
result = agent("Convert 100 meters to feet?")
print(result)

Thought: This is a length conversion from meters to feet. First, I need to get the conversion rate for meters to feet, so I can apply it to 100 meters.

Action: model_memory: meters to feet
PAUSE


In [75]:
result = model_memory("meters to feet")
print(result)

3.28084


In [76]:
next_prompt = "Observation: {}".format(result)
result = agent(next_prompt)
print(result)

Thought: Now that I have the conversion rate of 3.28084, I can apply this conversion rate to 100 meters to get the equivalent length in feet.

Action: apply_conversion: 3.28084, 100
PAUSE


In [77]:
result = apply_conversion("3.28084, 100")
result

'100.0 meters = 328.08 feet'

In [78]:
next_prompt = "Observation: {}".format(result)
agent(next_prompt)

'Thought: I have now successfully converted 100 meters to feet using the conversion rate.\n\nAnswer: 100 meters is equal to 328.08 feet.'

In [79]:
print(agent.state.messages)

[{'role': 'system', 'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\n    At the end of the loop you output an Answer\n    Use Thought to describe your thoughts about the question you have been asked.\n    Use Action to run one of the actions available to you - then return PAUSE.\n    Observation will be the result of running those actions.\n    \n    Your available actions are:\n    \n    model_memory:\n    e.g. model_memory: meters to feet\n    Get the conversion rate for a unit (e.g., meters to feet, kilometers to miles)\n\n    apply_conversion: 3.28084, 5\n    Applies the conversion rate to a value and returns a number\n    \n    No other actions are available to you.\n    \n    Example session #1:\n    Question: Convert 5 meters to feet?\n    \n    Thought: This is a length conversion from meters to feet. First, I need to get the conversion rate\n    Action: get_conversion_rate: meters to feet\n    PAUSE\n\n    You will be called again with this:\n    \n    Ob

## Running Everythin in the Loop

In [86]:
# Available actions mapping
KNOWN_ACTIONS = {
    "model_memory": model_memory,
    "apply_conversion": apply_conversion,
}

# Regular expression for parsing actions
ACTION_PATTERN = re.compile("^Action: (\w+): (.*)$")


def query(question: str, max_turns: int = 5) -> List[Dict[str, str]]:
    """
    Process a question through multiple turns of the ReAct loop.

    Args:
        question: The user's question
        max_turns: Maximum number of turns to process

    Returns:
        The complete conversation history
    """
    agent = create_agent()
    next_prompt = question

    for turn in range(max_turns):
        #print(f"INFO: turn {turn+1}")
        result = agent(next_prompt)
        #print(f"\nAgent Response:\n{result}")
        print(result)
        #print(f"INFO: end of preliminary result")
        # Look for actions in the response
        actions = [
            ACTION_PATTERN.match(a)
            for a in result.split("\n")
            if ACTION_PATTERN.match(a)
        ]
        
        if actions:
            print(f"INFO: Detected actions:{actions[0].groups()}")
            # Execute the action
            action, action_input = actions[0].groups()
            
            if action not in KNOWN_ACTIONS:
                raise ValueError(f"Unknown action: {action}: {action_input}")

            print(f"\n ---> Executing {action} with input: {action_input}")
            observation = KNOWN_ACTIONS[action](action_input)
            print(f"Observation: {observation}")
            next_prompt = f"Observation: {observation}"
        else:
            break

    return agent.state.messages

In [87]:
# Example 1: Length conversion
print("\nExample 1: Length Conversion")
print("=" * 50)
question = "Convert 10 meters to feet"
result = query(question)
print("\n" + "=" * 50 + "\n")


Example 1: Length Conversion
Thought: This is a length conversion from meters to feet. First, I need to get the conversion rate for meters to feet, which will allow me to convert 10 meters to feet.

Action: model_memory: meters to feet
PAUSE
INFO: Detected actions:('model_memory', 'meters to feet')

 ---> Executing model_memory with input: meters to feet
Observation: 3.28084
Thought: Now that I have the conversion rate of 3.28084, I can apply this conversion rate to 10 meters to find the equivalent length in feet.

Action: apply_conversion: 3.28084, 10
PAUSE
INFO: Detected actions:('apply_conversion', '3.28084, 10')

 ---> Executing apply_conversion with input: 3.28084, 10
Observation: 10.0 meters = 32.81 feet
Thought: I have now successfully converted 10 meters to feet using the conversion rate.

Answer: 10 meters is equal to 32.81 feet




In [88]:
# Example 2: Temperature conversion
print("\nExample 2: Temperature Conversion")
print("=" * 50)
question = "Convert 20°C to Fahrenheit"
result = query(question)
print("\n" + "=" * 50 + "\n")


Example 2: Temperature Conversion
Thought: This is a temperature conversion from Celsius to Fahrenheit. First, I need to get the conversion formula, which is typically a multiplication and addition operation. I will use the model_memory action to retrieve this formula.

Action: model_memory: celsius to fahrenheit
PAUSE
INFO: Detected actions:('model_memory', 'celsius to fahrenheit')

 ---> Executing model_memory with input: celsius to fahrenheit
Observation: 9/5,32
Thought: Now I can apply this formula to 20°C. The formula is (9/5)*C + 32, where C is the temperature in Celsius. I will use the apply_conversion action to perform this calculation.

Action: apply_conversion: 9/5, 32, 20
PAUSE
INFO: Detected actions:('apply_conversion', '9/5, 32, 20')

 ---> Executing apply_conversion with input: 9/5, 32, 20
Observation: 20.0°C = 68.00°F
Thought: The conversion has been successfully applied, and I have the result. Now, I can provide the final answer.

Answer: 20.0°C is equal to 68.00°F




In [89]:
# Example 3: Weight conversion
print("\nExample 3: Weight Conversion")
print("=" * 50)
question = "Convert 10 kilograms to pounds"
result = query(question)
print("\n" + "=" * 50 + "\n")


Example 3: Weight Conversion
Thought: This is a weight conversion from kilograms to pounds. First, I need to get the conversion rate
Action: model_memory: kilograms to pounds
PAUSE
INFO: Detected actions:('model_memory', 'kilograms to pounds')

 ---> Executing model_memory with input: kilograms to pounds
Observation: 2.20462
Thought: Now I can apply this conversion rate to 10 kilograms
Action: apply_conversion: 2.20462, 10
PAUSE
INFO: Detected actions:('apply_conversion', '2.20462, 10')

 ---> Executing apply_conversion with input: 2.20462, 10
Observation: 10.0 kilograms = 22.05 pounds
Thought: The conversion of 10 kilograms to pounds has been successfully calculated
Answer: 10 kilograms is equal to 22.05 pounds




In [338]:

# Example 4: Distance conversion
print("\nExample 4: Distance Conversion")
print("=" * 50)
question = "Convert 3 kilometers to miles"
result = query(question)
print("\n" + "=" * 50 + "\n")


Example 4: Distance Conversion
Thought: This is a length conversion from kilometers to miles. First, I need to get the conversion rate for kilometers to miles.

Action: get_conversion_rate: kilometers to miles
PAUSE

 ---> Executing get_conversion_rate with input: kilometers to miles
Observation: 0.621371
Thought: Now that I have the conversion rate, I can apply it to 3 kilometers to get the equivalent distance in miles.

Action: apply_conversion: 0.621371, 3
PAUSE

 ---> Executing apply_conversion with input: 0.621371, 3
Observation: 3.0 miles = 1.86 miles
Thought: It seems there was an error in the previous observation. However, I will proceed with the correct calculation. The correct conversion of 3 kilometers to miles using the conversion rate 0.621371 is actually 3 * 0.621371 = 1.864113 miles, which can be rounded to 1.86 miles.

Answer: 3 kilometers is equal to 1.86 miles.




# 🎮 Mission Accomplished!

**Your ReAct agent is now operational with capabilities to:**
- Navigate complex problems using structured reasoning cycles
- Connect with and utilize external tools
- Preserve dialogue history and context
- Break down and solve multi-stage challenges